In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

In [30]:
players = pd.read_csv('../data/raw/People.csv')
appearances = pd.read_csv('../data/raw/Appearances.csv')
batting = pd.read_csv('../data/raw/Batting.csv')
teams = pd.read_csv('../data/raw/Teams.csv')

In [55]:
# test = batting[batting['playerID'] == 'amezaal01']
# test.loc[(test.groupby(['playerID', 'yearID']).AB.rank(ascending=False) == 1.0), 'newteamID'] = test.teamID
# test.loc[(test.groupby(['playerID', 'yearID']).G.rank(ascending=False) == 1.0), 'newteamID'] = test.teamID

# test
# batting.loc[(batting.groupby(['playerID', 'yearID']).AB.rank(ascending=False) == 1.0)]

### Roll up appearances

In [72]:
# check_ = player_batting_appearances.query("playerID == 'martimi02' & yearID == 2016")
# appearances.groupby(['playerID', 'yearID'])['teamID'].count().sort_values(ascending=False)
appearances[appearances.playerID == 'drakeol01'].groupby(['playerID', 'yearID']).sum()

def main_position(X):
    positions = ['G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_dh']
    X_ = X.loc[:, X.columns.isin(positions)]
    main_pos = np.argmax(X_.values, axis=1)
    return main_pos

def appearances_player_year(data):
    cols = ['playerID', 'yearID', 'G_all', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_dh']
    df = data.groupby(['playerID', 'yearID'], as_index=False).sum()[cols]
    df['position_main'] = main_position(df)
    return df.loc[df['position_main'] > 0, ['playerID', 'yearID', 'position_main']]

positions_dict = {0: 'P', 1: 'C', 2: '1B', 3: '2B', 4: '3B', 5: 'SS', 6: 'LF', 7: 'CF', 8: 'RF', 9: 'DH'}
# X = appearances_player_year()
# main_position(X)

# X = appearances_player_year()

# X.shape

# check that each player year is unique
# X.groupby(['playerID', 'yearID']).count()[['G_all']].sort_values('G_all', ascending=False)


Need to aggregate hitting metrics up to team/year, exclude stint

In [ ]:
#     # get team with the most ab or games played
#     team = []
#     for index, player_year in data.groupby(['playerID', 'yearID']):
#         N = len(player_year)
        
#         # if there is only 1 record get the team and continue
#         if N == 1:
#             team.append(player_year['teamID'].values[0])
#             continue

#         max_G = player_year.G.max()
#         max_AB = player_year.AB.max()
        
#         # first check if the max ABs has a unique record and get the team
#         # otherwise get the team with the max games played
#         if len(player_year[player_year.AB == max_AB]['teamID']) > 1:
#             for i in range(N):
#                 team.append(player_year[player_year.G == max_G]['teamID'].values[0])
#         else:
#             for i in range(N):
#                 team.append(player_year[player_year.AB == max_AB]['teamID'].values[0])

### Roll up batting

In [56]:
def batting_player_year(data):
    """
    Args
    ----
        data (pd.DataFrame): a dataframe containing batting data
    
    Returns
    -------
        data (pd.DataFrame): a dataframe rolled up to the player year
        
    Some player years have multiple records since players can be traded over
    the course of the year and may have played for multiple teams. To rectify
    this the team that corresponded with the most ABs is chosen as the `main team`.
    In the case where both records have the same number of ABs for both teams we
    check the number of games played and choose the team with the most games played.
    """
    
    cols = ['playerID', 'yearID', 'teamID', 'G', 'AB', 'HR']
    data = data.groupby(['playerID', 'yearID', 'teamID'], as_index=False).sum()[cols]

    # data values must be sorted by playerID and yearID
    data.sort_values(by=['playerID', 'yearID'], inplace=True)

    # get team with most ABs or Gs
    team = []
    for index, player_year in data.groupby(['playerID', 'yearID']):
        N = len(player_year)
        
        # if there is only 1 record get the team and continue
        if N == 1:
            team.append(player_year['teamID'].values[0])
            continue

        max_G = player_year.G.max()
        max_AB = player_year.AB.max()
        
        # first check if the max ABs has a unique record and get the team
        # otherwise get the team with the max games played
        if len(player_year[player_year.AB == max_AB]['teamID']) > 1:
            for i in range(N):
                team.append(player_year[player_year.G == max_G]['teamID'].values[0])
        else:
            for i in range(N):
                team.append(player_year[player_year.AB == max_AB]['teamID'].values[0])
    
    data['team'] = team
    data = data.groupby(['playerID', 'yearID', 'team'], as_index=False).sum()[['playerID', 'yearID', 'team', 'AB', 'HR']]

    return data

In [57]:
X_b = batting_player_year(batting)

# check that each player year is unique
# X_b.groupby(['playerID', 'yearID', 'teamID']).count()[['G']].sort_values('G', ascending=False)

In [61]:
X_b.shape

(97992, 5)

In [27]:
print(X.columns)
print(X_b.columns)

Index(['playerID', 'yearID', 'position_main'], dtype='object')
Index(['playerID', 'yearID', 'team', 'AB', 'HR'], dtype='object')


In [62]:
X.shape

(54951, 3)

In [67]:
print(X_b.columns.values)

['playerID' 'yearID' 'team' 'AB' 'HR']


In [70]:
X_b.dtypes

playerID    object
yearID       int64
team        object
AB           int64
HR           int64
dtype: object

## Get full dataset

In [73]:
# clean up batting / appearance data sets
X_b = batting_player_year(batting)
X_a = appearances_player_year(appearances)

In [76]:
# join all the data
data = X_b \
    .merge(X_a, on=['playerID', 'yearID'], how='inner') \
    .merge(players[['playerID', 'nameFirst', 'nameLast', 'birthYear', 'debut']], on=['playerID'], how='inner') \
    .merge(teams[['yearID', 'teamID', 'park']], left_on=['yearID', 'team'], right_on=['yearID', 'teamID'])

data.drop(columns='teamID', inplace=True)

data['playerAge'] = data['yearID'] - data['birthYear']
data['yearsInMLB'] = data.groupby('playerID')['yearID'].rank(method='dense')
data['debut'] = pd.to_datetime(data['debut'])
data['playerFullName'] = data['nameFirst'] + ' ' + data['nameLast']

data = data[data['debut'].dt.year >= 2000].sort_values(['playerID', 'yearID'])

In [77]:
data.shape

(8597, 14)

In [80]:
data.dtypes

playerID                  object
yearID                     int64
team                      object
AB                         int64
HR                         int64
position_main              int64
nameFirst                 object
nameLast                  object
birthYear                float64
debut             datetime64[ns]
park                      object
playerAge                float64
yearsInMLB               float64
playerFullName            object
dtype: object

In [81]:
data.astype({'birthYear': np.int64, 'playerAge': np.int64, 'yearsInMLB': np.int64})

,playerID,yearID,team,AB,HR,position_main,nameFirst,nameLast,birthYear,debut,park,playerAge,yearsInMLB,playerFullName
448,abadan01,2001,OAK,1,0,2,Andy,Abad,1972,2001-09-10,Oakland Coliseum,29,1,Andy Abad
468,abadan01,2003,BOS,17,0,2,Andy,Abad,1972,2001-09-10,Fenway Park II,31,2,Andy Abad
1065,abercre01,2006,FLO,255,5,7,Reggie,Abercrombie,1980,2006-04-04,Dolphin Stadium,26,1,Reggie Abercrombie
1084,abercre01,2007,FLO,76,2,7,Reggie,Abercrombie,1980,2006-04-04,Dolphin Stadium,27,2,Reggie Abercrombie
1105,abercre01,2008,HOU,55,2,6,Reggie,Abercrombie,1980,2006-04-04,Minute Maid Park,28,3,Reggie Abercrombie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2279,zuninmi01,2014,SEA,438,22,1,Mike,Zunino,1991,2013-06-12,Safeco Field,23,2,Mike Zunino
2302,zuninmi01,2015,SEA,350,11,1,Mike,Zunino,1991,2013-06-12,Safeco Field,24,3,Mike Zunino
19332,zuninmi01,2016,SEA,164,12,1,Mike,Zunino,1991,2013-06-12,Safeco Field,25,4,Mike Zunino
35876,zuninmi01,2017,SEA,387,25,1,Mike,Zunino,1991,2013-06-12,Safeco Field,26,5,Mike Zunino
